### References
__Deep Learning - Goodfellow et al.__

__Backpropagation: Basic Theory - Rumelhart et al.__ 

When we use a feedforward neural network to accept an input $x$ and produce an output $\hat{y}$, information flows forward through the network. The inputs $x$ provide the initial information that then propagates up to the hidden units at each layer and finally produces $\hat{y}$. This is called forward propagation. During training, forward propagation can continue onward until it produces a scalar cost $J(\theta)$. The back-propagation algorithm __(Rumelhart et al., 1986a)__, allows the information from the cost to then flow backwards through
the network, in order to compute the gradient.

#### Theory of Backpropagation

The backprogation algorithm involves specifying a cost function and then modifying the weights iteratively according to the gradient of the cost function. The goal is to find the network which is the most likely explanation of the observed data. We can express this as attempting to maximize the term,

$$
P(N|D) = \frac{P(D|N)P(N)}{P(D)} 
$$

where $N$ represents the network(with all of the weights and biases specified), $D$ represents the observed data, and $P(D|N)$ is the probability that the network $N$ would have produced the observed data $D$. 

Simplifying the equation using logarithm, we get,

$$
ln(P(N|D)) = ln(P(D|N)) + ln(P(N)) - ln(P(D))
$$

as probability of the data is not dependent on the network, it is sufficient to maximize,

$$
ln(P(D|N)) + ln(P(N))
$$

Here the first term, $ln(P(D|N))$ is a measure of how well the network accounts for the data. We will first focus on this term as it is directly related to the performance of the network.

Simplifying further based on the fact that the data can be broken down into a set of observations, each, we will assume chosen independently of others, we get,

$$
\begin{align*}
ln(P(D|N)) = ln(\prod_{i} P(\langle \vec{x_i}, \vec{d_i} \rangle|N)) \\
= \sum_{i} ln(P(\langle \vec{x_i}, \vec{d_i} \rangle)|N) \\
= \sum_{i} ln(P(\vec{d_i}|\vec{x_i} \wedge N)) + \sum_{i} ln(P(\vec{x_i}))
\end{align*}
$$

Now, since we suppose that the event $\vec{x_i}$ does not depend on the network, the second term of the equation will not affect the determination of the optimal network. Therefore to maximize $ln(P(D|N))$ we only need to maximize $\sum_{i} ln(P(\vec{d_i}|\vec{x_i} \wedge N))$.

Upto now the process had been quite general, with the only assumption that obversed data points are chosen independent of each other. To carry out further analysis, several assumption have to be made:
1. The relation between $x_i$ and $d_i$ is not deterministic, instead, for any given $\vec{x_i}$, there is a distribution of possible values of $\vec{d_i}$.
2. The network is deterministic, so rather than predicting the actual outcome it will only attempt to predict the expected value of $\vec{d_i}$ given $\vec{x_i}$.

Thus the network output $\vec{y_i}$ is to be interpreted as the mean of the actual observed value.

__Rumelhart et al.__ studied the cases for when the form of the distribution is specified, the cases studied were,
1. Gaussian - For ordinary linear regression
2. Binomial - For binary classification
3. Multinomial - For multiple classification

Investigating all these cases led to a general finding, all of them end up with essentially the same learning rule in spite of different models. This occurs due to the fact that the probability distribution functions chosen all belong to the __exponential__ family of probability distributions. In addition to Gaussian/Normal, Binomial, this family of probability distributions include, the gamma distribution, the exponential distribution, the Poisson distribution, the negative binomial distribution and most other familiar probability distributions. 

The general form of the exponential family of probability distributions is,

$$
P(\vec{d_i}|\vec{x_i} \wedge N) = \exp(\sum_{i} \frac{(d_{i}\theta - B(\theta)) + C(\vec{d}\phi)}{a(\phi)})
$$

The result that we are most interested in is that for all members of the exponential family we get, 

$$
\frac{\partial C}{\partial \eta_j} \propto \frac{d_j - y_j}{var(y_j)}
$$

where $C$ is the cost function of the particular model. We then choose an output function one whose derivative with respect to $\eta$ is equal to the variance. For members of the exponential family of probability distributions we can always do this. 

Other import results from the paper include,

__Weight Decay__

$$
\frac{\partial C}{\partial w_{ij}} \propto \frac{1}{\sigma^{2}} w_{ij}
$$

__Weight Elimination__

$$
\frac{\partial C}{\partial w_{ij}} \propto - \frac{\sigma^{2}_{2}}{\sigma^{2}_{1}} \frac{w_{ij}}{(\sigma^{2}_{2} + w^{2}_{ij})^{2}}
$$

## Implementation of Backpropagation

We will describe how to compute the gradient $\nabla_{x} f(x,y)$ for an arbitrary function $f$, where $x$ is a set of variables that are inputs to the function whose derivatives are desired, and $y$ is an additional set of variables that are inputs to the function but whose derivatives are not required. 

In learning algorithms, the gradient we most often require is the gradient of the cost function with respect to the parameters, $\nabla_{\theta} J(\theta)$. Many machine learning tasks involve computing other derivatives, either as part of the learning process, or to analyze the learned model. The backpropagation algorithm can be applied to these tasks as well, and is not restricted to computing the gradient of the cost function with respect to the parameters.

The idea of computing derivatives by propagating information through a network is very general, and can be used to compute values such as the Jacobian of a function $f$ with multiple outputs. We restrict our description here to the most commonly used case where $f$ has a single output.

#### Mathematical Prerequisites

__Chain Rule__

Suppose that $y = g(x)$ and $z = f(g(x)) = f(y)$. Then the chain rule states that,

$$
\frac{dz}{dx} = \frac{dz}{dy} \frac{dy}{dx}
$$

Generalizing, suppose that $\vec{x} \in \R^{m}$, $\vec{y} \in \R^{n}$, $g$ maps from $\R^{m}$ to $\R^{n}$, and $f$ maps from $\R^{n}$ to $\R$. if $y = g(x)$ and $z = f(y)$, then,

$$
\frac{\partial z}{\partial x_{i}} = \sum_{j} \frac{\partial z}{\partial y_{j}} \frac{\partial y_{j}}{\partial x_{i}}
$$

### Writing a Neural Network from scratch